In [2]:
import re
import os
import glob
import numpy as np
import pandas as pd


infer_dir = "../results/inference"

# task_name = "okvqa"
# ds_split = "train"

# task_name = "mmmu"
# ds_split = "validation"

task_name = "mmmu_pro"
ds_split = "test"


model_names = ["llava-v1.6-vicuna-7b-hf", "llava-v1.6-vicuna-13b-hf",
               "Qwen2.5-VL-7B-Instruct", "InternVL2-8B",
               "deepseek-vl2-tiny", "deepseek-vl2-small"]


def extract_letter(text):
    match = re.search(r"\((\w)\)", text)
    return match.group(1) if match else ""

error_list = []
for mn in model_names:
    data_path = os.path.join(infer_dir, task_name, ds_split, f"{mn}_output.csv")
    data_df = pd.read_csv(data_path)

    arr_path = os.path.join(infer_dir, task_name, ds_split, f"{mn}_prob.npy")
    prob_arr = np.load(arr_path)

    start_chr = 'A'
    choices = []
    for i in range(prob_arr.shape[1]):
        choices.append(start_chr)
        start_chr = chr(ord(start_chr) + 1)

    prob_pred = []
    for i in np.argmax(prob_arr, axis=1):
        prob_pred.append(choices[i])
    prob_pred = np.array(prob_pred, dtype=str)

    generated_outputs = data_df["generated_outputs"].values

    extracted_outputs = []
    for output in generated_outputs:
        pred_txt = str(output)[:10].strip()
        if "\n" in pred_txt:
            pred_txt = pred_txt.split("\n")[1]
        if "(" in pred_txt or ")" in pred_txt:
            pred_txt = extract_letter(pred_txt)
        extracted_outputs.append(pred_txt[:1].upper())
    extracted_outputs = np.array(extracted_outputs)

    labels = data_df["answer"].values.astype(str) 
    if task_name == "mmmu_pro" and "llava" not in mn:
        extracted_outputs = np.delete(extracted_outputs, (1017), axis=0)
        prob_pred = np.delete(prob_pred, (1017), axis=0)
        labels = np.delete(labels, (1017), axis=0)

    errors = labels == extracted_outputs.astype(str)
    error_list.append(errors.astype(int))
    acc = np.mean(errors)

    print(prob_arr.shape)
    # print(mn, acc, np.mean(data_df["answer"].values.astype(str) == prob_pred))
    


(1591, 9)
(1591, 9)
(1592, 9)
(1592, 9)
(1592, 9)
(1592, 9)


In [18]:
import torch

exp_dict = torch.load("../results/ensemble/exp_result.pth")
error_arr = np.array(error_list)

logits = exp_dict["logits"]
ens_preds = logits[:, -9:].argmax(axis=1)
ens_err = (ens_preds == exp_dict["labels"]).astype(int)

In [19]:
import pandas as pd
import numpy as np


data_path = "../results/inference/ocr/train"

model_names = ["Qwen2.5-VL-7B-Instruct", "llava-v1.6-vicuna-7b-hf", "llava-v1.6-vicuna-13b-hf"]

model_outputs = []
answers = []
questions = []
for mn in model_names:
    data_df = pd.read_csv(f"{data_path}/{mn}_output.csv", index_col=0)
    model_outputs.append(data_df["generated_outputs"].values)
    answers = data_df["answer"].values
    questions = data_df["question"].values
model_outputs = np.array(model_outputs).T


In [20]:
model_outputs.shape, answers.shape, questions.shape

((3001, 3), (3001,), (3001,))

In [ ]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch


class MyDataset(Dataset):
    def __init__(self, tokenized_inputs, labels, global_attention_tokens=None, negative_inputs=None):
        self.tokenized_inputs = tokenized_inputs
        self.labels = labels
        self.global_attention_tokens = global_attention_tokens

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # input_ids = self.tokenized_inputs['input_ids'][idx]
        # attention_mask = self.tokenized_inputs['attention_mask'][idx]
        input_ids = torch.tensor(self.tokenized_inputs[idx].ids)
        attention_mask = torch.tensor(self.tokenized_inputs[idx].attention_mask)
        global_attentions = []
        start = False
        for i in input_ids:
            if start:
                if i == 50266:
                    start = False
                global_attentions.append(1)
            else:
                if i == 50265:
                    start = True
                global_attentions.append(0)
        global_attentions = torch.tensor(global_attentions)        # token_type_ids = self.tokenized_inputs['token_type_ids'][idx]
        label = self.labels[idx]
        return_dict = {'input_ids': input_ids,
                       "labels": label,
                       'attention_mask': attention_mask,
                       'global_attention_mask': global_attentions}

        return return_dict

